In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from urllib.parse import urlparse

In [4]:
url_base='https://www.cgmh.org.tw/tw/Systems/'
url_work_table='RecruitInfo/3?bulletinType=A&category=Z'
url=url_base+url_work_table
g=requests.get(url)
soup=BeautifulSoup(g.content, 'html.parser')

# job status
- display 
bulletinType=A job
- second setup
bulletinType=B interview 
- finally
bulletinType=C result

# work type
- ALL
category=Z
- Doctor
category=D
- Nurse
category=N
category=T

In [19]:
pages=soup.find_all('ul' ,class_="layout__pagination ul-reset")[0]
def get_pages(pages):
    return int(pages.find_all("li")[-2].text)

get_pages(pages)


13

In [50]:
def get_work_dead_line( soup):
    work_detail_web = soup.find('article').get_text().replace("\xa0","").replace('\n',"").replace("\u3000","")
    if '報名期限展延至' in work_detail_web:
        return re.findall("\d+年\d+月\d+日",work_detail_web.rsplit('報名期限展延至')[1].split("止")[0].replace(' ',''))[0]        
    elif '額滿' in work_detail_web:
        return '額滿為止'
    elif '招募合適人選為止' in work_detail_web:
        return '招募合適人選為止'
    elif '自即日起' in work_detail_web:
        return '自即日起'
    elif '即日起至' in work_detail_web:
        if "止" in work_detail_web.rsplit('即日起至')[1]:
            return re.findall("\d+年\d+月\d+日",work_detail_web.rsplit('即日起至')[1].split("止")[0].replace(' ',''))[0]
        elif "前" in work_detail_web:
            return re.findall("\d+年\d+月\d+日",work_detail_web.rsplit('即日起至')[1].split("前")[0].replace(' ',''))[0]
        elif "/" in work_detail_web.rsplit('即日起至')[1]:
            return re.findall("\d+/\d+/\d+",work_detail_web.rsplit('即日起至')[1].split("。")[0].replace(' ',''))[0]
    elif '即日起收件至' in work_detail_web:
        return work_detail_web.rsplit("即日起收件至")[1].split("止")[0].replace("：","")
    elif '前報名完成' in work_detail_web:
        return work_detail_web.rsplit("請於")[1].split("前")[0].replace("：","")    
    else:
        try:
            if '截止' in work_detail_web.rsplit("報名期限")[1]:
                return work_detail_web.rsplit("報名期限")[1].split("截止")[0].replace("：","")
            else:
                return work_detail_web.rsplit("報名期限")[1].split("。")[0].replace("：","")
        except:
            try:
                return re.findall("\d+年\d+月\d+日",work_detail_web)[0]
            except:
                return 'please check webpage'

In [22]:
def get_work_page(page):
    g=requests.get(url+'&page='+str(page))
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

In [16]:
def get_work_detail(link):
    g=requests.get(link)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup    

In [17]:
def get_work_table(soup,tables,work_table):
    for i, item in enumerate(tables):
        if item.find('a'): #過濾掉被刪除的文章
            s = item.find('a')
            url_base_website = urlparse(url)
            work_detail_link = url_base_website._replace(path=urlparse(s.get('href')).path).geturl()
            title = item.find_all('div')[1].string
            work_page_soup = get_work_detail(work_detail_link)
            dead_line = get_work_dead_line( work_page_soup )
            print('#{}召聘職稱: {} 期限: {}\n 連結：{}'.format(i+1, title, dead_line, work_detail_link ))
            work_table.append([i-2, title, dead_line, work_detail_link ])

In [51]:
work_table=[]
for _page in range(1,get_pages(pages)+1):
    print("page:",_page)
    soup_=get_work_page(_page)
    tables = soup_.find('div',class_="bg-grey pd100").find_all('ul')[-2].find_all('li')
    get_work_table(soup_,tables,work_table)

page: 1
#1召聘職稱: 轉公告長庚醫學科技股份有限公司資訊管理部招募資訊工程師展延報名期限，請查照。 期限: 2022年06月15日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202206020027?bulletinType=A&category=Z
#2召聘職稱: 長庚醫院行政中心工程企劃部企劃組招募一名營建工程師展延報名期限，請查照。 期限: 2022年06月15日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202206020024?bulletinType=A&category=Z
#3召聘職稱: 長庚醫療財團法人基隆暨情人湖院區護理部招募病患助理員，請 查照。 期限: 2022年6月16日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202206010061?bulletinType=A&category=Z
#4召聘職稱: 基隆長庚護理部長庚體系校園推甄(2022/6/9)甄試事宜公告，請 查照。 期限: 2022年6月9日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202206010060?bulletinType=A&category=Z
#5召聘職稱: 嘉義長庚幼兒園(籌備中)招募配膳助理員甄試結果公告 期限: 2022年5月11日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202206010059?bulletinType=A&category=Z
#6召聘職稱: 嘉義長庚幼兒園(籌備中)招募教保員甄試結果公告 期限: 2022年5月11日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202206010036?bulletinType=A&category=Z
#7召聘職稱: 高雄長庚麻醉科系招募定期契約人員，請 查照。 期限: 2022年06月16日
 連結：https://www.cgmh.org.tw/tw/Systems/

In [52]:

work_table=pd.DataFrame(work_table, columns=['no','召聘職稱','期限' ,'連結'])

print(work_table)

     no                                    召聘職稱                期限  \
0    -2  轉公告長庚醫學科技股份有限公司資訊管理部招募資訊工程師展延報名期限，請查照。       2022年06月15日   
1    -1    長庚醫院行政中心工程企劃部企劃組招募一名營建工程師展延報名期限，請查照。       2022年06月15日   
2     0        長庚醫療財團法人基隆暨情人湖院區護理部招募病患助理員，請 查照。        2022年6月16日   
3     1   基隆長庚護理部長庚體系校園推甄(2022/6/9)甄試事宜公告，請 查照。         2022年6月9日   
4     2               嘉義長庚幼兒園(籌備中)招募配膳助理員甄試結果公告        2022年5月11日   
..   ..                                     ...               ...   
122   0            高雄長庚紀念醫院招募照顧服務員(非受僱人員)，請 查照。       2022年12月31日   
123   1                   北院區臨床試驗中心誠徵臨床試驗研究助理一名              額滿為止   
124   2  辦理嘉義長庚醫院111年第一年暨第二年內科住院醫師招聘事宜，歡迎有意者報考。              額滿為止   
125   3                      林口長庚醫院兒童感染科徵博士後研究員       2022年06月30日   
126   4             公告嘉義長庚紀念醫院「非受僱-照顧服務員」招募甄試作業  即日起，至2022年06月30日   

                                                    連結  
0    https://www.cgmh.org.tw/tw/Systems/RecruitCont...  
1    https://www.cgmh.org.tw/tw/Systems/RecruitCont...  


In [38]:
re.findall("\d+年\d+月\d+日"," 報名期限：即日起至 2022年 6 月 30 日 截止".rsplit('即日起至')[1].split("止")[0].replace(' ',''))[0]

'2022年6月30日'

# interview

In [7]:
url = 'https://www.cgmh.org.tw/tw/Systems/RecruitInfo/3?bulletinType=B&category=Z&keyword='